In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/2020_11_20_23_59_11"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/2020_11_21_15_33_30"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/2020_11_22_15_07_50"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/2020_11_23_22_05_34"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/Elastix mask2020_11_25_01_04_36"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/ElastixNoCurv2020_11_27_20_57_40"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/ElastixNoCurv2020_11_28_16_04_41"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/NoElastixMask2020_11_29_14_53_29"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/_ElastixMask2020_11_29_20_41_48"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/_ElastixMaskNoCurv2020_11_30_00_34_21"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/_5ElastixMask2020_11_30_19_19_33"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/patchTest"
# resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/patchTest2"
resultsDir = "AAAAAAAAAAAAAAAAAAAAAAAAA/NsTest2020_12_06_16_35_43"

# Find all output folder
outputDirs = []
for fname in os.listdir(resultsDir):
    path = os.path.join(resultsDir, fname)
    if os.path.isdir(path):
        outputDirs.append(path)
        
# sort the list in order to match the testData.txt order
outputDirs.sort()
        
# Read all dice.txt files
diceData = []
for outputDir in outputDirs:
    diceFile = os.path.join(outputDir, "dice.txt")
    with open(diceFile, "r") as f:
        diceData.append(f.read())
        
# Find all the tests and initialize the dicts
dictSPBM = {}
dictSRC = {}
data = diceData[0].split("\n")
for line in data:
    col = line.find(":")
    if (line[:col]).isdigit():
        key = line[col+2:]
        dictSPBM[key] = []
        dictSRC[key] = []
        
# Populate the dicts
for data in diceData:
    lines = data.split("\n")
    for line in lines:
        col = line.find(":")
        if (line[:col]).isdigit():
            key = line[col+2:]
        elif line.startswith("SPBM: "):
            dictSPBM[key].append([float(num) for num in (line[6:]).strip('][').split(', ')])
        elif line.startswith("SRC: "):
            dictSRC[key].append([float(num) for num in (line[5:]).strip('][').split(', ')])
            
def getValues(key):
    values = key.split(',')
    string = "Atlases: "
    atlas  = int(values[0][len(string):])
    string = " P: "
    P = [int(num) for num in (values[1][len(string):]).strip('][').split(' ')]
    string = " N: "
    N = [int(num) for num in (values[2][len(string):]).strip('][').split(' ')]
    string = " lassoTol: "
    lassoTol = float(values[3][len(string):])
    return (atlas, P, N, lassoTol)

# Find all tests
atlases = []
Ps = []
Ns = []
lassoTols = []
for key, _ in dictSPBM.items():
    atlas, P, N, lassoTol = getValues(key)
    atlases.append(atlas)
    Ps.append(P)
    Ns.append(N)
    lassoTols.append(lassoTol)

def unique(a):
    indexes = np.unique(a, axis=0, return_index=True)[1]
    return [a[index] for index in sorted(indexes)]

atlases = unique(atlases)
Ps = unique(Ps)
Ns = unique(Ns)
lassoTols = unique(lassoTols)

if len(atlases) > 1:
    xlabels = atlases
    xlabel = "Number of atlases"
elif len(Ps) > 1:
    xlabels = Ps
    xlabel = "P"
elif len(Ns) > 1:
    xlabels = Ns
    xlabel = "N"
elif len(lassoTols) > 1:
    xlabels = lassoTols
    xlabel = "Lasso tollerance"

# Group per label
numOfLabels = 3

dataSPBM = []
dataSRC = []
for l in range(numOfLabels):
    dataSPBM.append([])
    dataSRC.append([])
    for _ in range(len(dictSPBM)):
        dataSPBM[l].append([])
        dataSRC[l].append([])
    
for index, (key, values) in enumerate(dictSPBM.items()):
    for v in values:
        for l in range(numOfLabels):
            dataSPBM[l][index].append(v[l])
        
for index, (key, values) in enumerate(dictSRC.items()):
    for v in values:
        for l in range(numOfLabels):
            dataSRC[l][index].append(v[l])


In [ ]:

for l in range(numOfLabels):
    plt.figure(dpi=150)
    plt.boxplot(dataSPBM[l], labels=xlabels)
    plt.title(f"SPBM {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()
    
    plt.figure(dpi=150)
    plt.boxplot(dataSRC[l], labels=xlabels)
    plt.title(f"SRC {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()

In [ ]:

for l in range(numOfLabels):
    plt.figure(dpi=150)
    plt.boxplot(dataSPBM[l], labels=xlabels)
    plt.title(f"SPBM {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()
    
    plt.figure(dpi=150)
    plt.boxplot(dataSRC[l], labels=xlabels)
    plt.title(f"SRC {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()

In [ ]:

for l in range(numOfLabels):
    plt.figure(dpi=150)
    plt.boxplot(dataSPBM[l], labels=xlabels)
    plt.title(f"SPBM {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()
    
    plt.figure(dpi=150)
    plt.boxplot(dataSRC[l], labels=xlabels)
    plt.title(f"SRC {l}")
    plt.xlabel(xlabel)
    plt.ylabel("Dice")
    plt.show()